In [2]:
import torch 
import pandas as pd
import numpy as np 


In [3]:
device = 'cuda' if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
from transformers import AutoTokenizer   , AutoModelForSeq2SeqLM

model_ = 'google/pegasus-cnn_dailymail'

tokenizer = AutoTokenizer.from_pretrained(model_)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_).to(device)

d:\Aiprojects\Textsummarization\text-summarization\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LOQ\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falli

In [13]:
import pandas as pd

splits = {'train': 'train.json', 'validation': 'val.json', 'test': 'test.json'}
df_train= pd.read_json("hf://datasets/nyamuda/samsum/" + splits["train"])
df_val= pd.read_json("hf://datasets/nyamuda/samsum/" + splits["validation"])
df_test= pd.read_json("hf://datasets/nyamuda/samsum/" + splits["test"])



In [14]:
df_train= df_train.drop(columns='id')
df_val = df_val.drop(columns='id')
df_test= df_test.drop(columns='id')

In [21]:
from datasets import load_dataset

ds = load_dataset("nyamuda/samsum")

In [22]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['id', 'summary', 'dialogue'],
        num_rows: 14732
    })
    validation: Dataset({
        features: ['id', 'summary', 'dialogue'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'summary', 'dialogue'],
        num_rows: 819
    })
})


In [23]:
def tokenization(example):
    input_encoding = tokenizer(example['dialogue'],max_length=1000,truncation=True)

    with tokenizer.as_target_tokenizer():
        output_encoding = tokenizer(example['summary'],max_length=150,truncation=True)

    return {'input_ids':input_encoding['input_ids'],
            'attention_mask':input_encoding['attention_mask'],
            'output_ids' :output_encoding['input_ids']
            }

In [24]:
ds=ds.map(tokenization,batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

d:\Aiprojects\Textsummarization\text-summarization\venv\Lib\site-packages\transformers\tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 819/819 [00:00<00:00, 5428.07 examples/s]


In [25]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'summary', 'dialogue', 'input_ids', 'attention_mask', 'output_ids'],
        num_rows: 14732
    })
    validation: Dataset({
        features: ['id', 'summary', 'dialogue', 'input_ids', 'attention_mask', 'output_ids'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'summary', 'dialogue', 'input_ids', 'attention_mask', 'output_ids'],
        num_rows: 819
    })
})